In [ ]:
import numpy as np
from collections import defaultdict
import json

In [176]:
draft_pick_movs = [[0.18897327219231208, 2.0010846899420685, 0.3732236160572243],
 [0.14727349526599293, 2.3890953884900448, 0.37877293453467575],
 [0.20993957998560997, 2.972927445404096, 0.29807102822209075],
 [0.2211666767279284, 3.449578532596238, 0.29588454235265493],
 [0.2204425469188902, 3.8786269293613316, 0.30687612009524046]]
age_shift ={19: 6.4,
 20: 6.7,
 21: 5.0,
 22: 4.4,
 23: 2.2,
 24: 2.0,
 25: 0.1,
 26: -0.1,
 27: -1.2,
 28: -1.4,
 29: -2.0,
 30: -2.7,
 31: -3.6,
 32: -3.7,
 33: -3.7,
 34: -4.7,
 35: -5.1}

max_shift = max(age_shift.values())


draft_salaries = defaultdict(lambda: 750,
{0: 7500,
 1: 6750,
 2: 6000,
 3: 5250,
 4: 4500,
 5: 4150,
 6: 3750,
 7: 3400,
 8: 3000,
 9: 2850,
 10: 2700,
 11: 2550,
 12: 2400,
 13: 2250,
 14: 2100,
 15: 1950,
 16: 1800,
 17: 1650,
 18: 1500,
 19: 1500,
 20: 1500,
 21: 1500,
 22: 1500,
 23: 1500,
 24: 1500,
 25: 1500,
 26: 1500,
 27: 1500,
 28: 1500,
 29: 1500,
})


iv = np.array([74.7 ,  7.92, 20.8 ])

def get_mov(x):
    offset = len(iv)//3
    tot = -2.2492
    for i in range(offset):
        tot += (iv[offset+i])*(np.tanh((x-iv[i])/iv[2*offset+i])+1.0)
    return tot

# estimated mov per ovr
o2m = {}
for i in np.arange(101):
    o2m[i] = get_mov(i)

# replacement level
r_lvl = -1.306

# draft model's baseline. 
# probably should always be above replacement
# not because they are, but because otherwise they're a pain
draft_lvl = o2m[0]

# salaries to mov
sA,sB = (3.15,0)

# expected mov, weight for players, weight for salary cap space
player_vs_cap = {
0 : [0, 1.0, 0] ,
1 : [0, 1.0, 0.9] ,
2 : [0, 1.0, 0.8] ,
3 : [0, 1.0, 0.6] ,
4 : [0, 1.0, 0.5] ,
}

RESIGN_CHANCE = 0.5


In [177]:
team = { 'p' : [(22,66,22000,2),(22,66,22000,2),(22,66,22000,2),(22,66,22000,2)], #(age,ovr,salary,years_left)
         'd' : [(1, 0, 8.6),(1, 0, -5.35),(2,0,3),(1,1,3),(2,1,3),(1,2,3),(2,2,3),(1,3,3),(2,3,3),(1,4,3),(2,4,3)] #(round,years_left,team_MOV)
       }
def get_team_value(team,sCap=90000,maxS=30000,minS=750,FAKE_RESIGN=True):
    sCapS = sCap/90000

    # turn mov into draft pick and future mov
    m2pos = lambda x: int(round(np.clip( 29/(1+np.exp(0.0036 - 0.368*(x))),0,29)))
    m2next = lambda year,mov: [1,0.5,0.25,0.08,0.03][year]*mov 

    # turn draft picks into specific predictions
    draft_picks = [ (yr,m2pos(m2next(yr,mov))+30*(rnd-1)) for rnd,yr,mov in team['d'] if yr < 5]
    # for this and the next 4 years
    pars = defaultdict(list)
    tss = defaultdict(int)

    for i in range(5):
        for age,ovr,con,yrl in team['p']:
            if yrl >= i: # are we on the team still?
                tss[i] += con
                ovr2=ovr
                for j in range(i):
                    ovr2+=age_shift.get(age+j,max_shift)
                pars[i].append(o2m[round(np.clip(ovr2,0,100))])
            elif FAKE_RESIGN:
                ovr2=ovr
                for j in range(i):
                    ovr2+=age_shift.get(age+j,max_shift)
                mov_p = o2m[round(np.clip(ovr2,0,100))]
                if mov_p > 0: # resign positive MOV contributors
                    est_con = min(1,(mov_p-r_lvl)/sA) * maxS 
                    tss[i] += est_con*RESIGN_CHANCE
                    pars[i].append(RESIGN_CHANCE*mov_p)  
        for yr,p in draft_picks:
            if yr+1 <= i: # if drafted
                dsal = sCapS*draft_salaries[p]
                tss[i] += dsal
                year_since_draft = i-(yr+1)
                x = draft_pick_movs[year_since_draft]
                pars[i].append(draft_lvl + x[1]*np.exp(-x[0]*(p**x[2])))
                #print(i, yr, p, pars[i][-1],dsal)

    # add salaries and player values
    pred_movs = []
    for i in range(5):
        play = [p for p in pars[i] if p >= r_lvl]
        lp = len(play)
        if lp < 10:
            play= play + (10-lp)*[r_lvl]
        play = sorted(play,reverse=True)[:10]
        #        play = [r_lvl+(p-r_lvl)*np.exp(-0.17330874*i) for i,p in enumerate(sorted(play,reverse=True)[:10])]
        play_s = sum(play)
        salary_s = tss[i] + (10-lp)*minS
        cap_space = (1.0/3.0)*np.maximum(sCap-salary_s,0)/sCap
        mov_from_cap = cap_space*sA

        x = player_vs_cap[i]
        p_mov = x[0] + x[1]*play_s + x[2]*mov_from_cap
        pred_movs.append(p_mov)
        #if i == 0:
        #print(i,lp,play_s,mov_from_cap)


    # turn mov into win probs
    cA,cB = 0.46421272651304596, -5.12021285460421
    # turn mov into 'make the finals'
    cA,cB = (0.42422076396359476, -3.94406073999211)

    win_p = [1.0/(1+np.exp(-mov*cA -cB)) for mov in pred_movs]

    # discount factor for the future, more uncertainty, less sure reward
    value = [wp*(0.8**(i)) for i,wp in enumerate(win_p)]
    #print(np.round(pred_movs,1))
    return sum(value)
get_team_value(team,90000,750)

1.3511371881589285

In [178]:
data = json.load(open('real_2020.json','rt',encoding='utf-8-sig'))
gA = {_['key']:_['value'] for _ in data['gameAttributes']}

In [179]:
team_players = defaultdict(list)
team_names = {}
season = gA['season']
sCap = gA['salaryCap']
minS = gA['minContract']
maxS = gA['maxContract']

team_injuries = defaultdict(list)
for p in data['players']:
    if p['tid'] >= 0:
        ovr = p['ratings'][-1]['ovr']
        age = season - p['born']['year']
        salary = p['contract']['amount']
        years_left = p['contract']['exp']-season
        team_players[p['tid']].append((age,ovr,salary,years_left))
        team_injuries[p['tid']].append(p['injury']['gamesRemaining'])


In [180]:
team_movs = defaultdict(float)

for t in data['teams']:
    tid = t['tid']
    team_names[t['tid']] = t['abbrev']
    for ts in t['stats']:
        if ts['playoffs']:
            continue
        current_mov = 0
        gp = ts['gp']+1e-9
        gl = 82-gp+1e-9
        if season == ts['season'] and not ts['playoffs'] and ts['gp']>0:
            mov = (ts['pts'] - ts['oppPts']) / ts['gp'];
            current_mov = mov
        estimated_mov = sum(sorted([o2m[_[1]]*(max(gl-i,0)/gl) for _,i in zip(team_players[tid],team_injuries[tid])])[-10:])
        team_movs[tid] = (gp/82)*current_mov + (gl/82)*estimated_mov    

In [181]:
team_picks = defaultdict(list)
for d in data['draftPicks']:
    mov = team_movs[d['originalTid']]
    tid = d['tid']
    rnd = d['round']
    yl = d['season']-season
    team_picks[tid].append((rnd,yl,mov))

In [182]:
teams_vals = []
for i in range(len(data['teams'])):
    #print(team_names[i])
    val = get_team_value({'p':team_players[i],'d':team_picks[i]},sCap,maxS,minS)
    teams_vals.append((val,team_names[i]))

In [183]:
for v,t in sorted(teams_vals)[::-1]:
    print(round(100*v,2),t)

100.9 NOL
57.96 LAC
56.99 LAL
45.43 BOS
43.98 HOU
41.01 MIL
34.98 TOR
24.7 POR
24.53 MIA
22.8 PHI
21.78 DEN
17.51 IND
16.05 BKN
13.7 ATL
10.42 DAL
9.57 SAC
8.14 MIN
8.03 MEM
7.76 CHI
6.6 ORL
5.04 PHO
4.14 OKC
3.98 GS
3.47 NYC
3.05 UTA
2.58 SA
1.78 CHA
1.61 WAS
1.03 CLE
0.37 DET


In [184]:
import random
deals = []
for i in range(1000):
    t1 = np.random.randint(30)
    t2 = np.random.randint(30)
    if t1 == t2:
        continue
    
    t1vo = get_team_value({'p':team_players[t1],'d':team_picks[t1]},sCap,maxS,minS)
    t2vo = get_team_value({'p':team_players[t2],'d':team_picks[t2]},sCap,maxS,minS)
    pn = np.random.randint(len(team_picks[t1]))
    picks = [_ for _ in team_picks[t1]]
    pick = picks[pn]        
    del picks[pn]

    for pi in range(len(team_players[t2])):
        local_p = [_ for _ in team_players[t2]]
        player = local_p[pi]
        del local_p[pi]
        t1v = get_team_value({'p':team_players[t1] + [player],'d':picks},sCap,maxS,minS)
        t2v = get_team_value({'p':local_p,'d':team_picks[t2] + [pick]},sCap,maxS,minS)
        if t1v > t1vo and t2v > t2vo:
            val = min((t1v-t1vo),(t2v-t2vo))
            deals.append((val,team_names[t1],pick,team_names[t2],player))

In [185]:
sorted(deals,reverse=True)

[(0.0205435674196226,
  'BOS',
  (1, 0, 5.526551762969072),
  'NOL',
  (29, 54, 13600, 0)),
 (0.01950555501744819,
  'IND',
  (1, 0, 1.8618297648976503),
  'MIA',
  (28, 53, 10250, 0)),
 (0.01810971660488825,
  'BOS',
  (1, 0, 5.526551762969072),
  'NOL',
  (29, 51, 4000, 1)),
 (0.01743236241671392,
  'LAC',
  (1, 1, 9.459320729575943),
  'MIA',
  (23, 50, 1600, 0)),
 (0.013160896589029726,
  'UTA',
  (1, 0, -0.2638671766618579),
  'DEN',
  (30, 55, 13650, 0)),
 (0.012740862058010727,
  'BOS',
  (1, 0, 5.526551762969072),
  'MIN',
  (33, 51, 15000, 1)),
 (0.012740862058010727,
  'BOS',
  (1, 0, 5.526551762969072),
  'MIN',
  (25, 51, 3300, 0)),
 (0.012675734269220529,
  'BOS',
  (2, 0, 5.526551762969072),
  'MIA',
  (23, 50, 1600, 0)),
 (0.010597509113084824,
  'IND',
  (1, 0, 1.8618297648976503),
  'MIA',
  (29, 54, 11650, 1)),
 (0.01003072245299362,
  'GS',
  (1, 0, -5.338114978144493),
  'POR',
  (31, 56, 24600, 0)),
 (0.009779809357546479,
  'BOS',
  (1, 0, 5.526551762969072),
  'C

In [186]:
import random
deals = []
for i in range(1000):
    t1 = np.random.randint(30)
    t2 = np.random.randint(30)
    if t1 == t2:
        continue
    
    t1vo = get_team_value({'p':team_players[t1],'d':team_picks[t1]},sCap,maxS,minS)
    t2vo = get_team_value({'p':team_players[t2],'d':team_picks[t2]},sCap,maxS,minS)
    
    pn = np.random.randint(len(team_players[t1]))
    players1 = [_ for _ in team_players[t1]]
    pick = players1[pn]        
    del players1[pn]

    for pi in range(len(team_players[t2])):
        local_p = [_ for _ in team_players[t2]]
        player = local_p[pi]
        del local_p[pi]
        t1v = get_team_value({'p':players1 + [player],'d':team_picks[t1]},sCap,maxS,minS)
        t2v = get_team_value({'p':local_p + [pick],'d':team_picks[t2]},sCap,maxS,minS)
        if t1v > t1vo and t2v > t2vo:
            val = (t1v-t1vo) + (t2v-t2vo)
            deals.append((val,team_names[t1],pick,team_names[t2],player))

In [187]:
for d in sorted(deals,reverse=True):
    if d[4][2]/d[2][2] < 1.25 and d[4][2]/d[2][2] > 1/1.25:
        print(d)

(0.11343474472320329, 'WAS', (27, 65, 28400, 3), 'DEN', (35, 60, 30350, 0))
(0.09004483512374996, 'NYC', (26, 58, 18900, 2), 'MIA', (34, 55, 17000, 0))
(0.0818469113864243, 'MIA', (36, 55, 13600, 2), 'BOS', (26, 62, 13000, 2))
(0.06660810957273844, 'DEN', (30, 53, 2050, 0), 'HOU', (36, 39, 2550, 0))
(0.05984323252061505, 'LAC', (23, 52, 7150, 3), 'SAC', (32, 56, 6850, 1))
(0.05283129302340683, 'LAC', (23, 48, 3750, 2), 'NOL', (29, 51, 4000, 1))
(0.04832099427511102, 'SAC', (25, 55, 6500, 1), 'LAC', (23, 52, 7150, 3))
(0.04832099427511102, 'LAC', (23, 52, 7150, 3), 'SAC', (25, 55, 6500, 1))
(0.03203553589181146, 'MIN', (25, 51, 3300, 0), 'LAC', (23, 48, 3750, 2))
(0.02860600398173363, 'TOR', (27, 58, 10500, 2), 'LAL', (33, 59, 11650, 1))
(0.02045374898401759, 'LAL', (34, 52, 2600, 1), 'PHO', (26, 55, 2700, 0))
(0.019691670552329565, 'POR', (28, 54, 5850, 1), 'BKN', (34, 52, 6750, 1))
(0.016044826622990144, 'MIA', (23, 50, 1600, 0), 'PHI', (23, 47, 1650, 1))
(0.016044826622990144, 'MIA',

In [188]:
import random
deals = set()
for i in range(1000):
    t1 = np.random.randint(30)
    t2 = np.random.randint(30)
    if t1 == t2:
        continue
    
    t1vo = get_team_value({'p':team_players[t1],'d':team_picks[t1]},sCap,maxS,minS)
    t2vo = get_team_value({'p':team_players[t2],'d':team_picks[t2]},sCap,maxS,minS)
    
    pn = np.random.randint(len(team_players[t1]))
    players1 = [_ for _ in team_players[t1]]
    pick = players1[pn]        
    del players1[pn]

    for pi in range(len(team_players[t2])):
        local_p = [_ for _ in team_players[t2]]
        player = local_p[pi]
        del local_p[pi]
        if not (pick[2]/player[2] < 1.25 and pick[2]/player[2] > 1/1.25):
            continue
        for p2 in range(len(team_picks[t2])):
            local_picks = [_ for _ in team_picks[t2]]
            pick2 = local_picks[p2]       
            del local_picks[p2]
            t1v1 = get_team_value({'p':players1 + [player],'d':team_picks[t1]},sCap,maxS,minS)

            t1v = get_team_value({'p':players1 + [player],'d':team_picks[t1] + [pick2]},sCap,maxS,minS)
            t2v = get_team_value({'p':local_p + [pick],'d':local_picks },sCap,maxS,minS)
            if t1v1 < t1vo and t1v > t1vo and t2v > t2vo:
                v1 = (t1v-t1vo)
                v2 = (t2v-t2vo)
                val = min( v1 , v2)
                deals.add((val,v1,v2,team_names[t1],pick,team_names[t2],player,pick2))
                

In [189]:
sorted(list(deals),reverse=True)

[(0.012596644157107512,
  0.013923702786432568,
  0.012596644157107512,
  'DEN',
  (29, 57, 13400, 2),
  'BKN',
  (27, 55, 11450, 2),
  (1, 0, 4.671793603770823)),
 (0.010795416932871907,
  0.010795416932871907,
  0.01133702583399717,
  'DAL',
  (27, 58, 4000, 2),
  'GS',
  (24, 51, 4800, 2),
  (1, 0, -5.338114978144493)),
 (0.01072875684439456,
  0.014283020357875098,
  0.01072875684439456,
  'MIA',
  (31, 67, 35200, 3),
  'BOS',
  (30, 66, 35200, 3),
  (1, 1, -1.7091913567911536)),
 (0.010325108677912781,
  0.010325108677912781,
  0.03360212373999705,
  'MIN',
  (25, 51, 3300, 0),
  'LAC',
  (23, 33, 3900, 3),
  (2, 0, 9.459320729575943)),
 (0.009471951782721769,
  0.009471951782721769,
  0.03424260288274028,
  'MIN',
  (25, 51, 3300, 0),
  'LAC',
  (23, 33, 3900, 3),
  (1, 1, 9.459320729575943)),
 (0.008751287354837045,
  0.008751287354837045,
  0.013496715668116444,
  'MIA',
  (31, 67, 35200, 3),
  'BOS',
  (30, 66, 35200, 3),
  (2, 0, 5.526551762969072)),
 (0.008737062252663852,
 

In [190]:
get_team_value({'p':local_p + [pick],'d':local_picks },sCap,maxS,minS,FAKE_RESIGN=False)

0.05224287773229583

In [191]:
get_team_value({'p':team_players[t2],'d':team_picks[t2]},sCap,maxS,minS,FAKE_RESIGN=False)

0.061518255996126946

In [192]:
team_names

{0: 'BOS',
 1: 'NYC',
 2: 'GS',
 3: 'SAC',
 4: 'LAL',
 5: 'DET',
 6: 'PHI',
 7: 'ATL',
 8: 'WAS',
 9: 'CHI',
 10: 'OKC',
 11: 'HOU',
 12: 'MIL',
 13: 'PHO',
 14: 'LAC',
 15: 'CLE',
 16: 'POR',
 17: 'UTA',
 18: 'BKN',
 19: 'SA',
 20: 'DEN',
 21: 'IND',
 22: 'DAL',
 23: 'NOL',
 24: 'MIA',
 25: 'ORL',
 26: 'MIN',
 27: 'TOR',
 28: 'MEM',
 29: 'CHA'}

In [193]:
team_test = {'d':team_picks[7],'p':team_players[7]}

In [194]:
get_team_value(team_test,sCap,maxS,minS,FAKE_RESIGN=True),get_team_value(team_test,sCap,maxS,minS,FAKE_RESIGN=False)

(0.13702942394618783, 0.03294134085176086)

In [195]:
team_test2 = {'d': [
  (2, 0, -5.418723344900641),
  (2, 0, 8.540340637164833),
  (1, 1, -5.418723344900641),
  (2, 1, -5.39282330007588),
  (2, 1, 3.853918443823184),
  (1, 2, -5.418723344900641),
  (2, 2, -5.418723344900641),
  (2, 2, -5.39282330007588),
  (1, 3, -5.418723344900641),
  (2, 3, -5.418723344900641),
  (2, 3, -7.864634755995118),
  (1, 4, -5.418723344900641),
  (2, 4, -5.418723344900641),
  (1, 5, -5.418723344900641),
  (2, 5, -5.418723344900641),
  (2, 5, -0.7486026698639017),
  (1, 6, -5.418723344900641),
  (2, 6, -5.418723344900641),
  (2, 6, -6.1728821801766705)],
 'p': [(31, 50, 13350, 2),
  (26, 58, 17500, 3),
  (32, 43, 19000, 0),
  (43, 36, 2500, 0),
  (25, 27, 1700, 1),
  (22, 31, 1550, 2),
  (23, 25, 950, 1),
  (21, 51, 4850, 3),
  (22, 56, 4250, 2),
  (22, 58, 6650, 2),
  (23, 52, 8050, 3),
  (23, 59, 4150, 1),
  (27, 39, 1500, 0),
  (26, 43, 2600, 0),
  (24, 47, 2350, 0),
  (25, 42, 1600, 0)]}

In [196]:
get_team_value(team_test2,sCap,maxS,minS,FAKE_RESIGN=True),get_team_value(team_test2,sCap,maxS,minS,FAKE_RESIGN=False)

(0.10275357016636127, 0.024511276637819886)

In [197]:
team_test = {'d':team_picks[2],'p':team_players[2]}
get_team_value(team_test,sCap,maxS,minS,FAKE_RESIGN=True),get_team_value(team_test,sCap,maxS,minS,FAKE_RESIGN=False)

(0.03981446155698568, 0.03730560201190117)

In [198]:
team_test2 = {'d': [(2, 0, 1.63714319636652),
  (2, 0, -0.541253135267356),
  (2, 1, 4.211933478418106),
  (1, 1, -6.1728821801766705),
  (1, 1, -3.690676899636947),
  (2, 1, -3.690676899636947),
  (1, 2, -6.1728821801766705),
  (2, 2, -6.1728821801766705),
  (2, 2, 6.384246070143772),
  (1, 3, -6.1728821801766705),
  (1, 5, -6.1728821801766705),
  (2, 5, -6.1728821801766705),
  (1, 6, -6.1728821801766705)],
 'p': [(24, 51, 4800, 2),
  (24, 55, 1650, 2),
  (25, 59, 29550, 3),
  (27, 22, 1650, 2),
  (21, 25, 3900, 3),
  (30, 59, 20200, 4),
  (30, 64, 38000, 4),
  (32, 68, 40250, 2),
  (26, 26, 1650, 2),
  (23, 46, 1800, 1),
  (27, 26, 100, 0),
  (20, 33, 1750, 3),
  (23, 38, 200, 0),
  (28, 38, 1850, 2),
  (23, 29, 1650, 2)]}
get_team_value(team_test2,sCap,maxS,minS,FAKE_RESIGN=True),get_team_value(team_test2,sCap,maxS,minS,FAKE_RESIGN=False)

(0.03529950226395019, 0.03358655472770039)